In [ ]:
from pathlib import Path
import os

DAY = "20251201"
Version = "v1"

PROJECT_ROOT = Path(os.getenv("LLMSC_ROOT", ".")).resolve()

DATA_DIR = Path(os.getenv("LLMSC_DATA_DIR", PROJECT_ROOT / "input")).resolve()
OUT_DIR  = Path(os.getenv("LLMSC_OUT_DIR",  PROJECT_ROOT / "runs" / f"{DAY}.{Version}")).resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
import numpy as np
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print(f"🔒 Random seed set to {RANDOM_SEED} for reproducibility.")

🔒 Random seed set to 42 for reproducibility.


In [ ]:
import google.generativeai as genai
import scanpy as sc
import numpy as np
import pandas as pd
import scipy
import os
import gc
import time

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ==========================================
# 2. Data Loading & Stratified Subsampling
# ==========================================
print("🚀 Building Pan-Cancer T-cell Atlas (ESCA + RC + PACA)...")

# 1. List of filenames for the dataset to be used
datasets = {
    "ESCA": "GSE156728_ESCA_10X.CD8.counts.txt.gz",
    "PACA": "GSE156728_PACA_10X.CD8.counts.txt.gz",
    "RC":   "GSE156728_RC_10X.CD8.counts.txt.gz",
}


# 2. Metadata loading
meta_path = DATA_DIR / "GSE156728_metadata.txt.gz"
print(f"Loading Metadata from {meta_path}...")
meta_df = pd.read_csv(meta_path, sep="\t", index_col=0)

# 3. Settings for stratified subsampling
N_PER_CLUSTER = 100

# 4. Processing
adatas_list = []

for cancer_type, filename in datasets.items():
    counts_path = DATA_DIR / filename
    print(f"\nProcessing {cancer_type} ({filename})...")

    try:
        counts = pd.read_csv(file_path, sep="\t", index_col=0)
        adata_temp = sc.AnnData(counts.T, dtype='float32')
        del counts
        gc.collect()

        common_cells = adata_temp.obs_names.intersection(meta_df.index)
        adata_temp = adata_temp[common_cells]
        adata_temp.obs = meta_df.loc[common_cells]
        adata_temp.obs['Cancer_Type'] = cancer_type

        sampled_indices = []
        clusters = adata_temp.obs['meta.cluster'].unique()

        for cl in clusters:
            cells_in_cluster = adata_temp.obs[adata_temp.obs['meta.cluster'] == cl].index

            if len(cells_in_cluster) > N_PER_CLUSTER:
                sampled = np.random.choice(cells_in_cluster, N_PER_CLUSTER, replace=False)
            else:
                sampled = cells_in_cluster

            sampled_indices.extend(sampled)

        adata_sub = adata_temp[sampled_indices].copy()

        print(f"  -> Original: {adata_temp.n_obs} cells")
        print(f"  -> Stratified: {adata_sub.n_obs} cells (Balanced)")

        adatas_list.append(adata_sub)

        del adata_temp
        gc.collect()

    except Exception as e:
        print(f"  ⚠️ Error loading {cancer_type}: {e}")

# 5. Concatenate
if len(adatas_list) > 0:
    print("\nMerging datasets...")
    adata = sc.concat(adatas_list, join='outer', fill_value=0)
    adata.obs_names_make_unique()

    print(f"✅ Pan-Cancer Atlas Built: {adata.n_obs} cells, {adata.n_vars} genes")
    print(f"   Cancer Types: {adata.obs['Cancer_Type'].unique()}")
else:
    raise ValueError("No data was loaded. Please verify the path.")

# 6. Preprocessing (Normalization & Log Transformation)
print("\nPreprocessing: Normalizing and Log-transforming...")

adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.layers["logcounts"] = adata.X.copy()

adata.obs['Sample_ID'] = adata.obs.index
print("✅ Data Preprocessing Complete.")

save_path = OUT_DIR / "cd8_benchmark_data.h5ad"
print(f"\n💾 Saving Benchmark Data to: {save_path}")
adata.write_h5ad(save_path)

meta_save_path = OUT_DIR / "cd8_benchmark_metadata.csv"
adata.obs.to_csv(meta_save_path)
print(f"📄 Metadata saved to: {meta_save_path}")

print("\nCell counts per cluster (Should be balanced approx ~300 if 3 datasets merged):")
print(adata.obs['meta.cluster'].value_counts().head(10))

🚀 Building Pan-Cancer T-cell Atlas (ESCA + RC + PACA)...
Loading Metadata from /input/GSE156728_metadata.txt.gz...

Processing ESCA (GSE156728_ESCA_10X.CD8.counts.txt.gz)...
  -> Original: 12526 cells
  -> Stratified: 1619 cells (Balanced)

Processing PACA (GSE156728_PACA_10X.CD8.counts.txt.gz)...
  -> Original: 5957 cells
  -> Stratified: 1350 cells (Balanced)

Processing RC (GSE156728_RC_10X.CD8.counts.txt.gz)...
  -> Original: 16544 cells
  -> Stratified: 1497 cells (Balanced)

Merging datasets...
✅ Pan-Cancer Atlas Built: 4466 cells, 24148 genes
   Cancer Types: ['ESCA' 'PACA' 'RC']

Preprocessing: Normalizing and Log-transforming...
✅ Data Preprocessing Complete.

💾 Saving Benchmark Data to: /runs/20251201.v1/cd8_benchmark_data.h5ad
📄 Metadata saved to: /runs/20251201.v1/cd8_benchmark_metadata.csv

Cell counts per cluster (Should be balanced approx ~300 if 3 datasets merged):
meta.cluster
CD8.c01.Tn.MAL          300
CD8.c02.Tm.IL7R         300
CD8.c03.Tm.RPS12        300
CD8.c04.T